In [38]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [39]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [40]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [41]:
!playwright install 

In [42]:
SEASONS=list(range(2023,2024))
SEASONS

[2023]

In [43]:
DATA_DIR= 'data2'
STANDINGS_DIR= os.path.join(DATA_DIR, 'standings')
SCORES_DIR= os.path.join(DATA_DIR, 'score')

In [44]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        print(f"Fetching {url}, Attempt {i}")
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url, timeout=60000)  # Increase the timeout to 60,000 ms (60 seconds)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:  # Correct the exception name
            print(f"Timeout error on {url}")
        else:
            break
    return html

In [45]:
async def scrape_season(season):
    url= f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html= await get_html(url, '#content .filter')
    
    soup=BeautifulSoup(html)
    links=soup.find_all('a')
    href=[l['href']for l in links]
    standings_pages= [f'https://www.basketball-reference.com{l}'for l in href]
    
    for url in standings_pages:
        save_path= os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url,'#all_schedule')
        with open (save_path, 'w+') as f:
            f.write(html)

In [46]:
standing_files= os.listdir(STANDINGS_DIR)

In [47]:
standing_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2015_games-march.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2014_games-june.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2015_games-january.html',
 'NBA_2015_games-june.html',
 'NBA_2017_games-february.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2015_games-february.html',
 'NBA_2018_games-april.html',
 'NBA_2014_games-march.html',
 'NBA_2014_games-february.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2014_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NB

In [48]:
#chat gpt
async def scrape_game(standing_files):
    with open(standing_files, 'r') as f:
        html= f.read()

    soup=BeautifulSoup(html)
    links= soup.find_all('a')
    hrefs=[l.get('href')for l in links]
    box_scores= [l for l in hrefs if l and "boxscore" in l and '.html' in l]
    box_scores= [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        if os.path.exists(save_path):
            continue

        html = await get_html(url, '#content')
        if not html:
            continue 

        with open(save_path, 'w+') as f:
            f.write(html)

In [50]:
for f in standing_files:
    filepath= os.path.join(STANDINGS_DIR, f)
    
    await scrape_game(filepath)

Fetching https://www.basketball-reference.com/boxscores/201501050BRK.html, Attempt 1
Mavericks vs Nets, January 5, 2015 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201501050CHI.html, Attempt 1
Rockets vs Bulls, January 5, 2015 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201501050MEM.html, Attempt 1
Knicks vs Grizzlies, January 5, 2015 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201501050MIN.html, Attempt 1
Nuggets vs Timberwolves, January 5, 2015 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201501060SAS.html, Attempt 1
Pistons vs Spurs, January 6, 2015 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201501070CHO.html, Attempt 1
Pelicans vs Hornets, January 7, 2015 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201501070CLE.html, Attempt 1
Rockets vs Cavaliers, Jan

CancelledError: 

In [31]:
standing_files= [s for s in standing_files if '.html' is s]

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-31-e0ef050f741e>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  standing_files= [s for s in standing_files if '.html' is s]


In [33]:
standing_files = [s for s in standing_files if '.html' == s]
standing_files

[]